In [1]:
!pip install  transformers datasets accelerate

In [2]:
!nvidia-smi

Sun Nov 17 11:16:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.216.01             Driver Version: 535.216.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:35:00.0 Off |                    0 |
| N/A   35C    P0              20W /  72W |      0MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
device ="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
model_name='google/bigbird-pegasus-large-bigpatent'
tokenizer=AutoTokenizer.from_pretrained(model_name)

In [5]:
from datasets import load_dataset
train_dataset=load_dataset("abisee/cnn_dailymail","3.0.0",split="train[:5%]")
test_dataset=load_dataset("abisee/cnn_dailymail","3.0.0",split="test[:50%]")

print(len(train_dataset),len(test_dataset))

14356 5745


In [6]:
train_dataset[0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [7]:
model=AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
model.gradient_checkpointing_enable()

In [8]:
# Tokenization function
def tokenize_function(batch):
    inputs = tokenizer(batch['article'], max_length=512, truncation=True, padding='max_length')
    outputs = tokenizer(batch['highlights'], max_length=128, truncation=True, padding='max_length')
    inputs['labels'] = outputs['input_ids']  # Add labels for training
    return inputs

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=test_dataset.column_names)

In [9]:
from transformers import DataCollatorForSeq2Seq
data_collator=DataCollatorForSeq2Seq(tokenizer,model=model)

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=10,
    logging_dir="./logs",
    fp16=True,
    seed=42,
    gradient_accumulation_steps=4
)


In [11]:
from transformers import Trainer
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer

/tmp/ipykernel_2440/3350367849.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(


In [12]:
trainer.train()

Attention type 'block_sparse' is not possible if sequence_length: 512 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Epoch,Training Loss,Validation Loss
0,1.501700,1.652016
1,1.444300,1.613090


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 256, 'num_beams': 5, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3588, training_loss=1.8525183684584023, metrics={'train_runtime': 4767.837, 'train_samples_per_second': 6.022, 'train_steps_per_second': 0.753, 'total_flos': 4.145225663722291e+16, 'train_loss': 1.8525183684584023, 'epoch': 1.999442741710783})

In [13]:
model.save_pretrained('pegasus_finetuned')

In [14]:
tokenizer.save_pretrained('tokenizer_finetuned')

('tokenizer_finetuned/tokenizer_config.json',
 'tokenizer_finetuned/special_tokens_map.json',
 'tokenizer_finetuned/tokenizer.json')

In [ ]:
#predicion
import torch
device="cuda" if torch.cuda.is_available() else "cpu"
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
model=AutoModelForSeq2SeqLM.from_pretrained('pegasus_finetuned').to(device)
tokenizer=AutoTokenizer.from_pretrained('tokenizer_finetuned')


article = """

The Thirsty Crow

On a hot summer day, a thirsty crow flew around in search of water. The sun blazed in the sky, and the dry air made the crow feel even thirstier. After searching for a long time, it finally spotted a pot under a tree.

The crow excitedly flew down and looked into the pot. To its disappointment, the water level was very low, and its beak could not reach the water. The crow thought for a while, wondering how to get the water.

Suddenly, it had an idea! It noticed small pebbles scattered around the pot. The crow began picking up the pebbles one by one with its beak and dropping them into the pot. Slowly, the water level started to rise.

The crow continued its efforts until the water level was high enough to drink. With great relief, the crow quenched its thirst and flew away happily.
"""

# Tokenize the input
inputs = tokenizer(
    article,
    max_length=512,
    truncation=True,
    return_tensors="pt"
).to(device)

#Generate the summary
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=200,
    min_length=30,
    length_penalty=10.0,
    num_beams=4,
    early_stopping=True
)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Summary:", summary)


Attention type 'block_sparse' is not possible if sequence_length: 177 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


Summary: He noticed small pebbles scattered around a pot under a tree. The crow picked up the pebbles one by one with its beak and dropped them into the pot until the water level was high enough to drink.
